In [ ]:
import os
import pandas as pd
import cv2
import tensorflow as tf
import numpy as np
import glob
from tqdm import tqdm
import re
from sklearn.model_selection import train_test_split
import pickle
import tensorflow as tf

In [ ]:
LABELS = ["ForwardFall", "BackwardFall", "LeftFall", "RightFall", "GetDown", "SitDown", "Walk"]
LABEL2INDEX = dict(zip(LABELS, range(len(LABELS))))
INDEX2LABEL = dict(zip(range(len(LABELS)), LABELS))

IMAGE_WIDTH = 32
IMAGE_HEIGHT = 32

In [ ]:
def get_all_paths(directory):
    files = []
    for root, _, filenames in os.walk(directory):
        for filename in filenames:
            if filename.endswith('.mp4') or filename.endswith('.MOV'):
                files.append(os.path.join(root, filename))
    return files

def get_subject(file):
    return re.findall("(Subject\d).*",file)[0]

In [ ]:
def count_frames(video_path):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error: Could not open video.")
        return -1

    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    return total_frames

In [ ]:
def count_all_frames(files):
  frames = 0
  for file in files:
    frame = count_frames(file)
    frames+=frame
  return frames

In [ ]:
directory =  'ViFam'
df = pd.DataFrame()

for label in LABELS:
  path = os.path.join(directory, label)
  files = sorted(get_all_paths(path))
  print(label)
  print(len(files))
  frames = count_all_frames(files)
  print(frames)
  sub_df = pd.DataFrame({"name": files})
  sub_df['label'] = label
  if len(df)==0:
      df = sub_df
  else:
      df = pd.concat([df, sub_df],axis=0)
df = df.sample(len(df))
X = df[["name"]]
y = df[["label"]]
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.4, random_state=42, stratify=y_val)

df_train = pd.concat([X_train, y_train], axis=1)
df_train.to_csv("df_train.csv", index=False)
df_val = pd.concat([X_val, y_val], axis=1)
df_val.to_csv("df_val.csv", index=False)
df_test = pd.concat([X_test, y_test], axis=1)
df_test.to_csv("df_test.csv", index=False)

ForwardFall
52
15478
BackwardFall
52
15525
LeftFall
50
15553
RightFall
49
15548
GetDown
72
15517
SitDown
65
15534
Walk
52
15600


In [ ]:
len(df_train), len(df_val), len(df_test)

(235, 94, 63)

# Create datasets

In [ ]:
def create_dataset(df, text: str):
    all_frames_count = 0
    for filename in tqdm(df.name.values):
        video = cv2.VideoCapture(filename)
        frames_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
        all_frames_count+=frames_count

    images = np.zeros((int(all_frames_count), IMAGE_HEIGHT, IMAGE_WIDTH, 1), dtype=np.float32)
    labels = np.zeros(int(all_frames_count))
    count = 0
    for index, row in tqdm(df.iterrows()):
        filename, label = row
        vid = cv2.VideoCapture(filename)
        success, frame = vid.read()
        while success:
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            image = frame
            frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
            frame = tf.keras.utils.img_to_array(frame)
            frame = tf.expand_dims(frame, 0)
            images[count] = frame
            labels[count] = LABEL2INDEX[label]
            count+=1
            success, frame = vid.read()
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    tf.data.Dataset.save(
      dataset, text, compression='GZIP'
  )
    with open(text + '/element_spec', 'wb') as out_:
        pickle.dump(dataset.element_spec, out_)

In [ ]:
df = df_train
text = "train"
create_dataset(df, text)

100%|██████████| 235/235 [00:02<00:00, 106.34it/s]
235it [18:19,  4.68s/it]


In [ ]:
df = df_test
text = "test"
create_dataset(df, text)

100%|██████████| 63/63 [00:01<00:00, 60.73it/s]
63it [04:07,  3.92s/it]


In [ ]:
df = df_val
text = "val"
create_dataset(df, text)

100%|██████████| 94/94 [00:01<00:00, 70.75it/s]
94it [07:23,  4.72s/it]


# Create datasets for 3D-CNN model

In [ ]:
def create_overlap_dataset(df: pd.DataFrame, img_width: int, img_height: int):
    """Tạo video frame từ các video có path nằm trong df. Mỗi video frame gồm 36 frames liên tiếp nhau."""

    video_frames = [] # list contain all video frames
    labels = [] # list contain labels of video frames

    for index, row in tqdm(df.iterrows()): # loop each video
        filename, label = row
        vid = cv2.VideoCapture(filename)
        end_frame = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
        success, frame = vid.read()
        frame_count = 0

        # save overlap video frames in 1 video
        clip_1 = []
        clip_2 = []
        clip_3 = []
        clip_4 = []
        clip_5 = []
        clip_6 = []
        clip_7 = []
        clip_8 = []
        clip_9 = []
        clip_10 = []
        clip_11 = []
        clip_12 = []
        while success:
            frame_count += 1
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(frame, (img_width, img_height), interpolation = cv2.INTER_NEAREST)

            clip_1.append(resized)
            if (frame_count > 3):
                clip_2.append(resized)
            if (frame_count > 6):
                clip_3.append(resized)
            if (frame_count > 9):
                clip_4.append(resized)
            if (frame_count > 12):
                clip_5.append(resized)
            if (frame_count > 15):
                clip_6.append(resized)
            if (frame_count > 18):
                clip_7.append(resized)
            if (frame_count > 21):
                clip_8.append(resized)
            if (frame_count > 24):
                clip_9.append(resized)
            if (frame_count > 27):
                clip_10.append(resized)
            if (frame_count > 30):
                clip_11.append(resized)
            if (frame_count > 33):
                clip_12.append(resized)

            if (frame_count % 36 == 0):
                video_frames.append(clip_1)
                labels.append(LABEL2INDEX[label])
                clip_1 = []
            if (frame_count % 36 == 3 and frame_count > 3):
                video_frames.append(clip_2)
                labels.append(LABEL2INDEX[label])
                clip_2 = []
            if (frame_count % 36 == 6 and frame_count > 6):
                video_frames.append(clip_3)
                labels.append(LABEL2INDEX[label])
                clip_3 = []
            if (frame_count % 36 == 9 and frame_count > 9):
                video_frames.append(clip_4)
                labels.append(LABEL2INDEX[label])
                clip_4 = []
            if (frame_count % 36 == 12 and frame_count > 12):
                video_frames.append(clip_5)
                labels.append(LABEL2INDEX[label])
                clip_5 = []
            if (frame_count % 36 == 15 and frame_count > 15):
                video_frames.append(clip_6)
                labels.append(LABEL2INDEX[label])
                clip_6 = []
            if (frame_count % 36 == 18 and frame_count > 18):
                video_frames.append(clip_7)
                labels.append(LABEL2INDEX[label])
                clip_7 = []
            if (frame_count % 36 == 21 and frame_count > 21):
                video_frames.append(clip_8)
                labels.append(LABEL2INDEX[label])
                clip_8 = []
            if (frame_count % 36 == 24 and frame_count > 24):
                video_frames.append(clip_9)
                labels.append(LABEL2INDEX[label])
                clip_9 = []
            if (frame_count % 36 == 27 and frame_count > 27):
                video_frames.append(clip_10)
                labels.append(LABEL2INDEX[label])
                clip_10 = []
            if (frame_count % 36 == 30 and frame_count > 30):
                video_frames.append(clip_11)
                labels.append(LABEL2INDEX[label])
                clip_11 = []
            if (frame_count % 36 == 33 and frame_count > 33):
                video_frames.append(clip_12)
                labels.append(LABEL2INDEX[label])
                clip_12 = []
            success, frame = vid.read()
        vid.release()

    video_frames = np.array(video_frames)
    print(video_frames.shape)
    labels = np.array(labels)
    print(labels.shape)

    return video_frames, labels

def save_3dcnn_dataset(dir: str, video_frames: np.array, labels: np.array):
    os.makedirs(dir, exist_ok = True)
    dataset = tf.data.Dataset.from_tensor_slices((video_frames, labels))
    tf.data.Dataset.save(dataset, dir, compression='GZIP')
    with open(dir + '/element_spec', 'wb') as out_:
        pickle.dump(dataset.element_spec, out_)

In [ ]:
os.mkdir("cnn3d")
video_frames, labels = create_overlap_dataset(df_train, IMAGE_WIDTH, IMAGE_HEIGHT)
save_3dcnn_dataset(os.path.join("cnn3d", "train"), video_frames, labels)
video_frames, labels = create_overlap_dataset(df_val, IMAGE_WIDTH, IMAGE_HEIGHT)
save_3dcnn_dataset(os.path.join(new_path, "val"), video_frames, labels)
video_frames, labels = create_overlap_dataset(df_test, IMAGE_WIDTH, IMAGE_HEIGHT)
save_3dcnn_dataset(os.path.join(new_path, "test"), video_frames, labels)